# Import libraries

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pylab
%matplotlib inline
from scipy.stats import norm

# Functions

In [ ]:
def rbe_wedenberg(dose, let, abx,q):
    """
    Wedenberg proton RBE model
    input parameters may be either numpy.array or scalars
    TODO: handle Cube() class directly
    :params dose: physical proton dose in [Gy]
    :params let: LETd in [keV/um] (protons only)
    :params abx: alpha_x / beta_x [Gy]
    :returns: RBE for the given parameters
    :ref: http://dx.doi.org/10.3109/0284186X.2012.705892
    """

    _apx = 1.000 + q * let / abx
    _sbpx = 1.0

    rbe = _rbe_apx(dose, _apx, _sbpx, abx)
    return rbe

In [ ]:
def _rbe_apx(dose, apx, sbpx, abx, dzero=0.0):
    """
    :params dose: proton dose      [Gy]
    :params apx: alpha_p / alpha_x [dimensionless] RBE_max = ap/ax when (dose -> 0 Gy)
    :params sbpx: beta_p / beta_x  [dimensionless] RBE_min = sqrt(bp/bx) when (dose -> inf Gy)
    :params abx: alpha_x / beta_x  [Gy]
    :params dzero: what to return in case of dose is zero (which would cause division by zero)
    """

    _rbe = 1.0 / (2.0 * dose)
    if hasattr(_rbe, '__iter__'):
        _rbe[_rbe == np.inf] = dzero
    else:
        if _rbe == np.inf:
            return dzero
    delta = abx * abx + 4. * abx * apx * dose + 4. * sbpx * sbpx * dose * dose
    delta *= (delta > 0)
    _rbe *= (np.sqrt(delta) - abx)
    return _rbe

In [ ]:
style_param = {'legend.fontsize': '20',
         'xtick.direction' : 'in',  
         'ytick.direction' : 'in', 
         'xtick.top' : True, 
         'figure.figsize': (10,6),
         'axes.labelsize': '28',
         'axes.titlesize':'28',
         'xtick.labelsize':'24',
         'ytick.labelsize':'24',
         'xtick.major.pad':'22',
         'ytick.major.pad':'22'}
    
pylab.rcParams.update(style_param)

In [ ]:
abx=2.

# Open data

In [ ]:
open_fname = os.path.join('tmp','distrib_q.h5')
df_q = pd.read_hdf(open_fname, 'data_1')

In [ ]:
open_fname = os.path.join('data','sobp_shieldhit','output')
dose = os.path.join(open_fname,'dose.dat')
let=os.path.join(open_fname,'dlet.dat')

In [ ]:
dozym = os.path.join('data','sobp_dosimetry','doz_sobp.xlsx')
df_dozym = pd.read_excel(dozym,names=['depth','dose','depth2','dose2'])

In [ ]:
df_dose=pd.read_table(dose,names=['z','dose','err'],sep='\s+')
df_let=pd.read_table(let,names=['z','let','err'],sep='\s+')

In [ ]:
# LET and normalized dose in water phantom
dose_max=df_dose.dose[df_dose.dose>0.95*df_dose.dose.max()].mean()
df= pd.DataFrame(dtype='float')

#normalize dose to 2[Gy]
df["dose"]=df_dose['dose']*2/dose_max
df["let"]=df_let['let']
df["z"]=df_dose['z']

In [ ]:
#z regresji liniowej - notatniki 02a i 02b
mean_Wed, std_Wed = 0.4335, 0.0301
mean_WedA, std_WedA = 0.389, 0.0218

In [ ]:
q_WA = np.random.normal(loc=mean_WedA,scale=std_WedA,size=50000)
q_W = np.random.normal(loc=mean_Wed,scale=std_Wed,size=50000)

In [ ]:
mean_W, down_W, up_W=  q_W.mean(), np.quantile(q_W,0.075), np.quantile(q_W,0.925)
mean_WA, down_WA, up_WA=  q_WA.mean(), np.quantile(q_WA,0.075), np.quantile(q_WA,0.925)
mean_WB, down_WB, up_WB = df_q.q.mean(), df_q.q.quantile(0.075), df_q.q.quantile(0.925)

In [ ]:
limit=df_dose.z[df_dose.dose<=df_dose.dose.max()*0.01].iloc[0]

In [ ]:
fig,ax = plt.subplots(figsize=[12, 8])

ax.set_xlabel("Zasięg [cm]",size=28, labelpad = 28)
ax.set_ylabel("Dawka [Gy]",size=28, labelpad = 28)

ax.plot(df.z, df.dose,'green',markersize=1.2, label = "Symulacja Monte Carlo SHIELD-HIT12A")
ax.plot(df_dozym.depth2,df_dozym.dose2,'red',markersize=1.2, label = "Pomiary dozymetryczne")

ax.grid()
ax.minorticks_on()
ax.grid(which='minor', linestyle=':', linewidth='0.2', color='k')
ax.tick_params(axis='both', which='major', labelsize=22)
ax.set_xlim(2,limit)
ax.set_ylim(0,2.5)

ax.legend(loc="center",bbox_to_anchor=(0.45, 0.1), borderaxespad=0.,fontsize=24)

In [ ]:
d_mean_dozym = df_dozym.dose2[(df_dozym.depth2>=5.5)&(df_dozym.depth2<=8.5)].mean()

In [ ]:
d_mean = df.dose[(df.z>=5.5)&(df.z<=8.5)].mean()

In [ ]:
#range for dosimetry
df.z[(df.z>8.5)&(df.dose<=d_mean*0.8)].min()

In [ ]:
(d_mean-d_mean_dozym)/d_mean_dozym*100

In [ ]:
fig,ax = plt.subplots(figsize=[12, 8])

ax2=ax.twinx()

ax.plot(df.z, df.dose,'green',markersize=1.2, label = "Dawka")
ax2.plot(df.z, df.let,'k.',markersize=1.2, label = "LET")

ax.set_xlabel("Zasięg [cm]",size=28, labelpad = 22)
ax.set_ylabel("Dawka [Gy]",size=28, labelpad = 22)
ax2.set_ylabel("LET "+r"$\;[keV \cdot \mu m^{-1}]$",size=34,labelpad = 22,rotation=90)

ax.grid()
ax.minorticks_on()
ax.grid(which='minor', linestyle=':', linewidth='0.2', color='k')
ax.tick_params(axis='both', which='major', labelsize=22)
ax2.tick_params(axis='both', which='major', labelsize=22)

#ax.set_title("SHIELDHIT-12A simulation")
ax.set_xlim(2,limit)
ax2.set_xlim(2,limit)
ax.set_ylim(0.,3.5)
ax2.set_ylim(0,14)

ax.legend(loc="center",bbox_to_anchor=(0.15, 0.9), borderaxespad=0.,fontsize=24)
ax2.legend(loc="center",bbox_to_anchor=(0.135, 0.78), borderaxespad=0.,fontsize=24)

In [ ]:
#min , max and mean let in SODP
df.let[(df.z >=5.5) & (df.z<=8.5)].min(),df.let[(df.z >=5.5) & (df.z<=8.5)].max(),df.let[(df.z >=5.5) & (df.z<=8.5)].mean()

In [ ]:
#LET in the center of SOBP 
df.let[(df.z >=7.)][700]

# Wedenberg RBE distribution for LET=2.

In [ ]:
tmp_WB = rbe_wedenberg(2.0,3.,abx, df_q )['q']
tmp_WA = pd.DataFrame(rbe_wedenberg(2.0,3., abx, q_WA))[0]
tmp_W = pd.DataFrame(rbe_wedenberg(2.0,3., abx, q_W))[0]

In [ ]:
WA_text = "poszerzony model Wedenberg A \n"+str(np.round(tmp_WA.mean(),3))+"+- "+str(np.round(1.44*tmp_WA.std(),4)) +' ('+str(np.round(1.44*tmp_WA.std()/tmp_WA.mean()*100,2))+'%)'

W_text = "model Wedenberg \n"+str(np.round(tmp_W.mean(),3))+"+- "+str(np.round(1.44*tmp_W.std(),4)) +' ('+str(np.round(1.44*tmp_W.std()/tmp_W.mean()*100,2))+'%)'

WB_text = "poszerzony model Wedenberg B \n"+str(np.round(tmp_WB.mean(),3))+"+- "+str(np.round(1.44*tmp_WB.std(),4)) +' ('+str(np.round(1.44*tmp_WB.std()/tmp_WB.mean()*100,2))+'%)'

In [ ]:
fig,ax = plt.subplots(figsize = (8,6))

tmp_W.plot.kde(ax=ax,color='r')
tmp_WA.plot.kde(ax=ax,color='blue')
tmp_WB.plot.kde(ax=ax,color='green')

ax.axvline(x=1.1, label='RBE = 1.1'.format(1.1),c='k',ls='--',lw=1.2)

ax.set_title("")
ax.set_xlabel("\nRBE [-]")
ax.set_ylabel("Rozkład\n prawdopodobieństwa [-]\n")

ax.grid()
ax.minorticks_on()
ax.grid(which='minor', linestyle=':', linewidth='0.2', color='k')

ax.legend([W_text,WA_text,WB_text],loc='upper center', bbox_to_anchor=(0.5, 1.55),fancybox=True, shadow=True,ncol=1)

In [ ]:
#tmp=pd.DataFrame(list(zip(tmp_W,tmp_WA,tmp_WB)),
#                          columns=['rbe'])
#tmp.to_csv("results/figure_5_9")
#fig.savefig(fname="results/figure_5_9.png",dpi= 700)

# our RBE distribution

In [ ]:
dfr = (df.reset_index(inplace=False))

In [ ]:
#optional : n=q.size-1
n=df_q.size-1

In [ ]:
dfr =dfr.append([dfr]*n)

In [ ]:
dfr=(dfr.reset_index(inplace=False))[['z','let', 'dose']]

In [ ]:
dfr=dfr.sort_values(by=['z', 'let','dose'])

In [ ]:
dfr.set_index(['z'],inplace=True) 

In [ ]:
for name, group in dfr.groupby('z'):
    dfr.loc[(name), "q"] = df_q.values[:n+1]

In [ ]:
for name, group in dfr.groupby('z'):
    DOSE=group.dose.values
    LET=group.let.values
    Q=group.q.values  
    dfr.loc[(name), "rbe"] = rbe_wedenberg(DOSE, LET, abx, Q)

In [ ]:
dfr.reset_index(inplace=True)

In [ ]:
df["wb_rbe_med"]=dfr.groupby('z').rbe.median().values
df["wb_rbe_mean"]=dfr.groupby('z').rbe.mean().values
df["wb_rbe_down"]=dfr.groupby('z').rbe.quantile(0.075).values
df["wb_rbe_up"]=dfr.groupby('z').rbe.quantile(0.925).values

In [ ]:
#RBE for q=0.434 from Wedenberg 85% CI
df['w_rbe_mean'] = rbe_wedenberg(df.dose, df.let, abx, mean_W)
df['w_rbe_down'] = rbe_wedenberg(df.dose, df.let, abx, down_W)
df['w_rbe_up'] = rbe_wedenberg(df.dose, df.let, abx, up_W)

# Biological Dose in SOBP

In [ ]:
figure2_df= pd.DataFrame()

In [ ]:
df["Distance_cm"] = df.z
df["Ext_Model_median"] = df.wb_rbe_med*df.dose
df["Ext_Model_mean"] = df.wb_rbe_mean*df.dose
df["Ext_Model_quantile_down"] = df.wb_rbe_down*df.dose
df["Ext_Model_quantile_up"] = df.wb_rbe_up*df.dose

df["Wedenberg"] = df.w_rbe_mean*df.dose
df["Wedenberg_quantile_down"] = df.w_rbe_down*df.dose
df["Wedenberg_quantile_up"] = df.w_rbe_up*df.dose

df["Physical__Dose"] = df.dose
df["Dose_for_RBE_1_1"] = df.dose*1.1


In [ ]:
fig,ax = plt.subplots(figsize=[12, 8])

ax2=ax.twinx()
ax.set_xlabel("\nZasięg [cm]")
ax.set_ylabel("Dawka biologiczna [Gy(RBE)]\n")
ax2.set_ylabel("\nLET "+r"$\;[keV \cdot \mu m^{-1}]$",rotation=90)

ax.set_xticks(np.arange(3, limit, 1.0))

ax2.plot(df.z, df.let,'k.',markersize=1.2, label = "LET")

#ax.plot(rbe.z, rbe.dose,c='k',label="Dose [Gy]")
ax.plot(df.Distance_cm,df.Wedenberg,'red', label = "model Wedenberg")
ax.plot(df.Distance_cm,df.Ext_Model_mean,'green',markersize=0.2, label= "poszerzony model Wedenberg B")
ax.plot(df.Distance_cm, df.Dose_for_RBE_1_1,'blue',markersize=0.1, label = "Dawka dla RBE 1.1") 

ax.scatter(df.Distance_cm,df.Ext_Model_quantile_down, s=0.1,c='green')
ax.scatter(df.Distance_cm,df.Ext_Model_quantile_up, s=0.1,c='green')

ax.scatter(df.Distance_cm,df.Wedenberg_quantile_down,s=0.1,c='red')
ax.scatter(df.Distance_cm,df.Wedenberg_quantile_up,s=0.1,c='red')


ax.set_ylim(0,4)
ax2.set_ylim(0,16)

ax.set_xlim(4,limit)
ax2.set_xlim(4,limit)

ax.grid()
ax.minorticks_on()
ax.grid(which='minor', linestyle=':', linewidth='0.2', color='k')
ax.tick_params(axis='both', which='major')

ax.axvspan(7.5,9, alpha=0.3, color='gray')

#ax.legend(loc="center",bbox_to_anchor=(0.33, 0.9), borderaxespad=0.,fontsize=20)
#ax2.legend(loc="center",bbox_to_anchor=(0.88, 0.95), borderaxespad=0.,fontsize=20)

ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.2),fancybox=True, shadow=True,ncol=2)

ax2.legend(loc='upper center', bbox_to_anchor=(0.4, 1.),fancybox=True, shadow=True,ncol=1)

In [ ]:
fig,ax = plt.subplots(figsize=[12, 8])

ax2=ax.twinx()
ax.set_xlabel("\nZasięg [cm]")
ax.set_ylabel("Dawka biologiczna [Gy(RBE)]\n")
ax2.set_ylabel("\nLET "+r"$\;[keV \cdot \mu m^{-1}]$",rotation=90)

ax.set_xticks(np.arange(3, limit, 1.0))

ax2.plot(df.z, df.let,'k.',markersize=1.2, label = "LET")

#ax.plot(rbe.z, rbe.dose,c='k',label="Dose [Gy]")
ax.plot(df.Distance_cm,df.Wedenberg,'red', label = "model Wedenberg")
ax.plot(df.Distance_cm,df.Ext_Model_mean,'green',markersize=0.2, label= "poszerzony model Wedenberg B")
ax.plot(df.Distance_cm, df.Dose_for_RBE_1_1,'blue',markersize=0.1, label = "Dawka dla RBE 1.1") 

ax.scatter(df.Distance_cm,df.Ext_Model_quantile_down, s=0.1,c='green')
ax.scatter(df.Distance_cm,df.Ext_Model_quantile_up, s=0.1,c='green')

ax.scatter(df.Distance_cm,df.Wedenberg_quantile_down,s=0.1,c='red')
ax.scatter(df.Distance_cm,df.Wedenberg_quantile_up,s=0.1,c='red')



ax.set_ylim(0,4)
ax2.set_ylim(0,16)

ax.set_xlim(7.5,9)
ax2.set_xlim(7.5,9)

start, end = ax.get_xlim()
stepsize=0.25
ax.xaxis.set_ticks(np.arange(start, end+stepsize, stepsize))


ax.grid()
ax.minorticks_on()
ax.grid(which='minor', linestyle=':', linewidth='0.2', color='k')
ax.tick_params(axis='both', which='major')

#ax.legend(loc="center",bbox_to_anchor=(0.33, 0.9), borderaxespad=0.,fontsize=20)
#ax2.legend(loc="center",bbox_to_anchor=(0.88, 0.95), borderaxespad=0.,fontsize=20)


In [ ]:
#mean doses in SOBP for 3 models
m_dose_W=df.Wedenberg[(df.Distance_cm >=5.5) &(df.Distance_cm <=8.5)].mean()
m_dose_WB=df.Ext_Model_mean[(df.Distance_cm >=5.5) &(df.Distance_cm <=8.5)].mean()
m_dose_C=df.Dose_for_RBE_1_1[(df.Distance_cm >=5.5) &(df.Distance_cm <=8.5)].mean()
m_dose_W,  m_dose_WB, m_dose_C

In [ ]:
#mean upper doses in SOBP for Wedenberg and Extended models
up_dose_W=df.Wedenberg_quantile_up[(df.Distance_cm >=5.5) &(df.Distance_cm <=8.5)].mean()
up_dose_WB=df.Ext_Model_quantile_up[(df.Distance_cm >=5.5) &(df.Distance_cm <=8.5)].mean()
up_dose_W, up_dose_WB

In [ ]:
#mean error in SOBP [Gy]
(up_dose_W-m_dose_W), (up_dose_WB-m_dose_WB)

In [ ]:
#mean error in SOBP [%]
(up_dose_W-m_dose_W)/m_dose_W*100, (up_dose_WB-m_dose_WB)/m_dose_WB*100

In [ ]:
#differences in mean doses in SOBP
(m_dose_W-m_dose_C)/m_dose_C *100,(m_dose_WB-m_dose_C)/m_dose_C *100,(m_dose_W-m_dose_WB)/m_dose_WB *100

In [ ]:
m_dose_W-m_dose_C,m_dose_WB-m_dose_C,m_dose_W-m_dose_WB

In [ ]:
#range for 3 models
range_W=df.Distance_cm[(df.Distance_cm >=8.5) & (df.Wedenberg <= m_dose_W*0.8)].min()
range_WB=df.Distance_cm[(df.Distance_cm >=8.5) & (df.Ext_Model_mean <= m_dose_WB*0.8)].min()
range_C=df.Distance_cm[(df.Distance_cm >=8.5) & (df.Dose_for_RBE_1_1 <= m_dose_C*0.8)].min()
range_C, range_W, range_WB

In [ ]:
    #differences in ranges  [cm]
(range_WB-range_C),(range_W-range_C),(range_W-range_WB)

In [ ]:
#differences in ranges [%]
(range_WB-range_C)/range_C*100,(range_W-range_C)/range_C*100,(range_W-range_WB)/range_WB*100

In [ ]:
#doses in distal edge of SOBP for range = clinical range
d_dose_W=df.Wedenberg[df.Distance_cm ==range_C].values
d_dose_WB=df.Ext_Model_mean[df.Distance_cm ==range_C].values
d_dose_C=df.Dose_for_RBE_1_1[df.Distance_cm ==range_C].values
d_dose_W, d_dose_WB, d_dose_C

In [ ]:
#differences in doses in distal [Gy]
(d_dose_W-d_dose_C),(d_dose_WB-d_dose_C),(d_dose_W-d_dose_WB)

In [ ]:
#upper doses in distal edge of SOBP for range = clinical range
up_d_dose_W=df.Wedenberg_quantile_up[df.Distance_cm ==range_C].values
up_d_dose_WB=df.Ext_Model_quantile_up[df.Distance_cm ==range_C].values

In [ ]:
#differences in doses in distal SOBP [%]
(d_dose_W-d_dose_C)/d_dose_W*100,(d_dose_WB-d_dose_C)/d_dose_W *100,(d_dose_W-d_dose_WB)/d_dose_WB *100

In [ ]:
(d_dose_W-d_dose_C),(d_dose_WB-d_dose_C),(d_dose_W-d_dose_WB)

In [ ]:
#error in distal SOBP in one point [Gy]
(up_d_dose_W-d_dose_W), (up_d_dose_WB-d_dose_WB)

In [ ]:
#error in distal SOBP in one point [%]
(up_d_dose_W-d_dose_W)/d_dose_W*100, (up_d_dose_WB-d_dose_WB)/d_dose_WB*100

# IN SOBP

In [ ]:
plateau = 2.005
middle = 7.005
distal = 8.665

In [ ]:
def dose_in_sobp(range_insobp, title, lim1,lim2):
    fig,ax = plt.subplots(figsize = (10,6))

    df_W = pd.DataFrame(rbe_wedenberg(np.float64(df[df.z==range_insobp].dose.values), 
                        df[df.z == range_insobp].let.values, 
                        abx, q_W)*df[df.z == range_insobp].dose.values)
    
    df_W.plot.kde(ax=ax,color='red')
        
    df_WA = pd.DataFrame(rbe_wedenberg(np.float64(df[df.z==range_insobp].dose.values), 
                        df[df.z == range_insobp].let.values, 
                        abx, q_WA)*df[df.z == range_insobp].dose.values)
    
    df_WA.plot.kde(ax=ax,color='blue')

    df_WB = rbe_wedenberg(np.float64(df[df.z==range_insobp].dose.values), 
                        df[df.z == range_insobp].let.values, 
                        abx, df_q.q)*df[df.z == range_insobp].dose.values
    
    df_WB.plot.kde(ax=ax,color='green')
   
    #ax.axvline(figure2_df[figure2_df.Distance_cm == range_insobp].Dose_for_RBE_1_1.values, c='black',ls='-.')
    
    W_text =   "model Wedenberg\n" +str(np.round(df_W.mean()[0],3)) + "+-" + str(np.round(1.44*df_W.std()[0],3)) +" (" +  str(np.round(1.44*df_W.std()[0]/df_W.mean()[0]*100,2)) + "%)"
    WA_text =   "poszerzony model Wedenberg A\n" +str(np.round(df_WA.mean()[0],3)) + "+-" + str(np.round(1.44*df_WA.std()[0],3)) +" (" +  str(np.round(1.44*df_WA.std()[0]/df_WA.mean()[0]*100,2)) + "%)"
    WB_text =   "poszerzony model Wedenber B\n" +str(np.round(df_WB.mean(),3)) + "+-" + str(np.round(1.44*df_WB.std(),3)) +" (" +  str(np.round(1.44*df_WB.std()/df_WB.mean()*100,2)) + "%)"

    Clinic = "Dawka dla RBE = 1.1"
    
    print((df_W.mean()-df_WB.mean())/df_W.mean()*100)
    print((df_WB.mean()-df[df.Distance_cm == range_insobp].Dose_for_RBE_1_1.values)/df_WB.mean()*100)
    print((df_W.mean()-df[df.Distance_cm == range_insobp].Dose_for_RBE_1_1.values)/df_W.mean()*100)

    ax.set_xlabel("\nDawka biologiczna [Gy(RBE)]")
    ax.set_ylabel("Rozkład \nprawdopodobieństwa\n"+r"$[Gy(RBE)^{-1}]$")
    ax.set_title(title)
    ax.grid()
    ax.minorticks_on()
    ax.grid(which='minor', linestyle=':', linewidth='0.2', color='k')
    ax.set_xlim(lim1,lim2)
    
    ax.set_ylim(0,None)
    ax.legend([W_text,WA_text,WB_text, Clinic],loc='upper center', bbox_to_anchor=(1, 1),fancybox=True, shadow=True,ncol=1)

In [ ]:
dose_in_sobp(plateau, "Obszar wejściowy",1.,1.6)

In [ ]:
dose_in_sobp(middle, "Środek SOBP",2.1,2.7)

In [ ]:
dose_in_sobp(distal, "Dystalna część SOBP",2.3,2.9)

In [ ]:
fig,ax = plt.subplots(figsize=[12, 8])

ax2=ax.twinx()
ax.set_xlabel("\nZasięg [cm]")
ax.set_ylabel("Dawka biologiczna [Gy(RBE)]\n")
ax2.set_ylabel("\nLET "+r"$\;[keV \cdot \mu m^{-1}]$",rotation=90)

ax.set_xticks(np.arange(1, limit, 1.0))

ax2.plot(df.z, df.let,'k.',markersize=1.2, label = "LET")

#ax.plot(rbe.z, rbe.dose,c='k',label="Dose [Gy]")
ax.plot(df.Distance_cm,df.Wedenberg,'red', label = "model Wedenberg")

ax.plot(df.Distance_cm,df.Ext_Model_mean,'green',markersize=0.2, label= "poszerzony model Wedenberg B")
ax.plot(df.Distance_cm, df.Dose_for_RBE_1_1,'blue',markersize=0.1, label = "Dawka dla RBE 1.1") 

ax.scatter(df.Distance_cm,df.Ext_Model_quantile_down, s=0.1,c='green')
ax.scatter(df.Distance_cm,df.Ext_Model_quantile_up, s=0.1,c='green')

ax.scatter(df.Distance_cm,df.Wedenberg_quantile_down,s=0.1,c='red')
ax.scatter(df.Distance_cm,df.Wedenberg_quantile_up,s=0.1,c='red')


ax.set_ylim(0,4)
ax2.set_ylim(0,16)

ax.set_xlim(1,limit)
ax2.set_xlim(1,limit)

ax.grid()
ax.minorticks_on()
ax.grid(which='minor', linestyle=':', linewidth='0.2', color='k')
ax.tick_params(axis='both', which='major')

ax.axvline(plateau,  color='black',ls=':', lw=5)
ax.text(2.2, 3.2, 'A\nObszar\nwejściowy',fontsize=22)

ax.axvline(middle,  color='black',ls=':', lw=5)
ax.text(5.8, 3.2, 'B\nŚrodek\nSOBP',fontsize=22)


ax.axvline(distal,  color='black',ls=':', lw=5)
ax.text(7.2, 3., 'C\nDystalna\nczęść\nSOBP',fontsize=22)

#ax.legend(loc="center",bbox_to_anchor=(0.33, 0.9), borderaxespad=0.,fontsize=20)
#ax2.legend(loc="center",bbox_to_anchor=(0.88, 0.95), borderaxespad=0.,fontsize=20)

ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.2),fancybox=True, shadow=True,ncol=2)

ax2.legend(loc='upper center', bbox_to_anchor=(0.9, 1.13),fancybox=True, shadow=True,ncol=1)